In [ ]:
import os
from cellxgene_mods import CxG_API


CxG_API.config() # set env='dev' if working with dev portal

**Specify the Collection to upload to**<br>
If a Revision, use the Revision ID, not the Published ID

In [ ]:
collection_id = ''

**List the existing Datasets in the Collection**

In [ ]:
coll_datasets = {d['dataset_id']:d['title'] for d in CxG_API.get_collection(collection_id)['datasets']}
coll_datasets

**Set the directory that the files to upload are in**

In [ ]:
directory = os.path.expanduser('~/Downloads/')

**List certain files in that directory**<br>
The curation_qa notebook saves files with a `_revised.h5ad` suffix

In [ ]:
for f in os.listdir(directory):
    if f.endswith('_revised.h5ad'):
        print(f)

**Fill in file name:Dataset ID pairs**<br>
If adding a dataset, rather than replacing an existing dataset, use `new` for the Dataset ID

In [ ]:
datasets = {
    directory + 'my_matrix_revised.h5ad': 'new'
}

**Ensure that each file exists and each Dataset exists or is `new`**

In [ ]:
for f,d in datasets.items():
    if not os.path.exists(f):
        print('Invalid file:' + f)
    if d != 'new':
        if d not in coll_datasets:
            print('Invalid dataset:' + d)
        if list(datasets.values()).count(d) > 1:
            print('Repeated dataset:' + d)

**Upload each Dataset**

In [ ]:
for f,d in datasets.items():
    if d == 'new':
        d = CxG_API.create_dataset(collection_id)
    CxG_API.upload_local_datafile(f, collection_id, d)